<a href="https://colab.research.google.com/github/AhmedKhaledAK/chest-x-rays-diagnosis/blob/master/Project2_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Import necessary libraries**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import pickle 
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

**Data augmentation methods**




In [ ]:
def horizontal_flip(img):
  out = cv2.flip(img, 1)
  return out    
    
def vertical_flip(img):
  out = cv2.flip(img, 0)
  return out
  
def rotation(img):
  out = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
  return out 

**Read images from directory, augment them and save the augmented images into the directory**


In [ ]:
def augment_images(dir_path):
  for i in range(1, 1201):
    # read image from directory
    image = cv2.imread(dir_path.format(i), cv2.IMREAD_GRAYSCALE)

    # augment image
    if i < 401:
      aug_image = horizontal_flip(image)
    elif i < 801:
      aug_image = vertical_flip(image)
    else:
      aug_image = rotation(image)

    # save augmented image to directory
    cv2.imwrite(dir_path.format(i+1200), aug_image)

**Augment the images in all 3 directories**

In [ ]:
augment_images('/content/drive/MyDrive/Kaggle/COVID-19 Radiography Database/COVID/COVID ({}).png')
augment_images('/content/drive/MyDrive/Kaggle/COVID-19 Radiography Database/NORMAL/NORMAL ({}).png')
augment_images('/content/drive/MyDrive/Kaggle/COVID-19 Radiography Database/Viral Pneumonia/Viral Pneumonia ({}).png')

**Model Test functions**

In [ ]:
def report(y_pred,y_test):
  print("Confusion_matrix")
  classes=[0,1,2]
  print(confusion_matrix(y_test, y_pred, labels=classes))
  print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_test, y_pred)))
  print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
  print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
  print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))
  print('\nClassification Report\n')
  print(classification_report(y_test, y_pred, target_names=['Covid', 'Normal', 'Viral Pneumonia']))

def test_model(test_data_generator,y_test):
  y_pred = model.predict(test_data_generator)
  y_hat=[]
  for i in range(len(y_pred)):
    max_number = max(y_pred[i])
    index_of_maximum = np.where(y_pred[i] == max_number)
    y_hat.append(index_of_maximum[0][0])
  report(y_hat,y_test)  

**DataGenerator class**

In [ ]:
import time
class DataGenerator(keras.utils.Sequence):

  def __init__(self, list_IDs, labels, batch_size, dim, n_classes, convolutional, n_channels=3, shuffle=False):
    self.list_IDs = list_IDs
    self.labels = labels
    self.batch_size = batch_size
    self.dim = dim
    self.n_classes = n_classes
    self.n_channels = n_channels
    self.shuffle = shuffle
    self.convolutional = convolutional
    self.on_epoch_end()

  def __len__(self):
    # Number of batches per epoch
    return int(np.ceil(len(self.list_IDs) / self.batch_size))

  def __getitem__(self, index):
    # Generate one batch of data

    # generate indexes of the batch
    if(((index+1)*self.batch_size) < len(self.list_IDs)):
      indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
    else:
      indexes = self.indexes[index*self.batch_size : len(self.list_IDs)]
    
    # find list of IDs corresponding to these indexes
    list_IDs_temp = [self.list_IDs[k] for k in indexes]
    
    # generate data
    X, y = self.__data_generation(list_IDs_temp,len(indexes))
    
    return X, y

  def __data_generation(self, list_IDs_temp,size_of_batch):
    # Generates data

    # if CNN, data dimensions are (batch_size, n, n, n_channels)
    if self.convolutional:
      X = np.empty((size_of_batch, *self.dim, self.n_channels))
    else:
      X = np.empty((size_of_batch, self.dim*self.n_channels))

    y = np.empty((size_of_batch), dtype=int)

    # generate data
    for i, ID in enumerate(list_IDs_temp):
      if ID <= 2400:
        # load image (ID) from COVID folder
        image = cv2.imread("/content/drive/MyDrive/Kaggle/COVID-19 Radiography Database/COVID/COVID ({}).png".format(ID))
      elif ID <= 4800:
        # load image (ID-2400) from NORMAL folder
        image = cv2.imread("/content/drive/MyDrive/Kaggle/COVID-19 Radiography Database/NORMAL/NORMAL ({}).png".format(ID-2400))
      else:
        # load image (ID-4800) from Viral Pneumonia folder
        image = cv2.imread("/content/drive/MyDrive/Kaggle/COVID-19 Radiography Database/Viral Pneumonia/Viral Pneumonia ({}).png".format(ID-4800))

      image = cv2.resize(image, (256, 256))
      arr = tf.keras.applications.mobilenet_v2.preprocess_input(np.array(image))

      if self.convolutional:
        arr = arr.reshape(256,256,3)
      else:
        arr = arr.flatten()

      X[i,] = arr
      y[i] = self.labels[ID-1]

    return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

  def on_epoch_end(self):
    # Update indexes after each epoch
    self.indexes = np.arange(len(self.list_IDs))
    if self.shuffle == True:
        np.random.shuffle(self.indexes)

In [ ]:
@tf.function
def train_step(x, y,loss_fn,optimizer,train_acc_metric):
  # open a gradienttape to record operations done during forward pass
  # enables auto-differentiation
  with tf.GradientTape() as tape:
    # run forward pass of layer
    logits = model(x, training=True)

    # compute loss value for this batch
    loss_value = loss_fn(y, logits)
  
  # use gradient tape to get gradients of trainable variables wrt loss
  gradients = tape.gradient(loss_value, model.trainable_weights)

  # run one step of gradient descent (update value of weights to minimize loss)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))

  # update training metric
  train_acc_metric.update_state(y, logits)

  return loss_value

In [ ]:
@tf.function
def validation_step(x, y,val_acc_metric):
    val_logits = model(x, training=False)
    # update validation metrics
    val_acc_metric.update_state(y, val_logits)

In [ ]:
def training_loop(num_epochs, batch_size, training_generator, validation_generator):
  # instantiate an optimizer (stochastic gradient descent, learning rate = 0.001)
  optimizer = keras.optimizers.Adam(learning_rate=0.0001)

  # instantiate a loss function (categorical crossentropy)
  loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

  # Prepare the metrics
  train_acc_metric = keras.metrics.CategoricalAccuracy()
  val_acc_metric = keras.metrics.CategoricalAccuracy()

  # prepare training dataset
  batch_size = batch_size

  epochs = num_epochs
  num_batches = training_generator.__len__()

  for epoch in range(epochs):
    print("\n\nStart of epoch ", epoch)
    Total_loss = 0
    # iterate over all batches in dataset
    for i in range(num_batches):
      x_batch_train, y_batch_train = training_generator.__getitem__(i)

      Total_loss += train_step(x_batch_train, y_batch_train,loss_fn,optimizer,train_acc_metric)
    
    Total_loss = Total_loss/num_batches
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))
    print("Loss value over epoch: %.4f" % (float(Total_loss),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for i in range(validation_generator.__len__()):
      x_batch_val, y_batch_val = validation_generator.__getitem__(i)
      validation_step(x_batch_val, y_batch_val,val_acc_metric)

    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    

**Dataset consists of 7200 images with IDs 1-7200, and a corresponding label vector**

In [ ]:
# Data = 7200 images
data = np.arange(1, 7201)

# Create label vector
label_vector = np.empty((7200))

for i in range(0, 2400):
  label_vector[i] = 0

for i in range(2400, 4800):
  label_vector[i] = 1

for i in range(4800, 7200):
  label_vector[i] = 2


**Split data into train, validation and test sets to get a 60:20:20 ratio**

In [ ]:
# split data into train and test (80:20)
X_train, X_test, Y_train, Y_test = train_test_split(data, label_vector, test_size=0.2, random_state=1, stratify=label_vector)

# split train data into train and validation (75:25)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.25, random_state=1, stratify=Y_train)

(4320,)
(1440,)
(1440,)


In [ ]:
partition = {
    "train": X_train,
    "validation": X_validation,
    "test": X_test
}

**Create a fully connected neural network and train it on our dataset**


**Load Train and Validation and Test Set For Fully Connected Models**


In [ ]:
training_generator = DataGenerator(partition['train'], label_vector, 32, 256*256, 3, False)
validation_generator = DataGenerator(partition['validation'], label_vector, 32, 256*256, 3, False)
test_generator = DataGenerator(partition['test'], label_vector, 32, 256*256, 3, False)

**Few Layers Model**

In [ ]:
# Create model
model = Sequential()
model.add(Dense(512,input_dim=256*256*3, activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 512)               100663808 
_________________________________________________________________
dense_16 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_17 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 99        
Total params: 100,838,467
Trainable params: 100,838,467
Non-trainable params: 0
________________________________________

**Training Model**

In [ ]:
# Train model on dataset
#model.fit_generator(generator=training_generator,validation_data=validation_generator,use_multiprocessing=True, workers=6,epoch)
#model.fit_generator(generator=training_generator,validation_data=validation_generator,epochs=20)
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.8220
Loss value over epoch: 0.5409
Validation acc: 0.8861


Start of epoch  1
Training acc over epoch: 0.8836
Loss value over epoch: 0.3766
Validation acc: 0.8597


Start of epoch  2
Training acc over epoch: 0.9111
Loss value over epoch: 0.2527
Validation acc: 0.9007


Start of epoch  3
Training acc over epoch: 0.9250
Loss value over epoch: 0.2179
Validation acc: 0.8958


Start of epoch  4
Training acc over epoch: 0.9438
Loss value over epoch: 0.1667
Validation acc: 0.9014


Start of epoch  5
Training acc over epoch: 0.9463
Loss value over epoch: 0.1597
Validation acc: 0.9035


Start of epoch  6
Training acc over epoch: 0.9667
Loss value over epoch: 0.0971
Validation acc: 0.9014


Start of epoch  7
Training acc over epoch: 0.9708
Loss value over epoch: 0.0808
Validation acc: 0.9146


Start of epoch  8
Training acc over epoch: 0.9681
Loss value over epoch: 0.0959
Validation acc: 0.9118


Start of epoch  9
Training acc over epoch: 0.9590
Los

**Testing Model**

In [ ]:
#test_generator = DataGenerator(partition['test'], label_vector, 32, 256*256, 3, False, shuffle=False)
test_model(test_generator, Y_test)

confusion_matrix
[[462   3  15]
 [ 11 417  52]
 [ 15  19 446]]

Accuracy: 0.92

Weighted Precision: 0.92
Weighted Recall: 0.92
Weighted F1-score: 0.92

Classification Report

                 precision    recall  f1-score   support

          Covid       0.95      0.96      0.95       480
         Normal       0.95      0.87      0.91       480
Viral Pneumonia       0.87      0.93      0.90       480

       accuracy                           0.92      1440
      macro avg       0.92      0.92      0.92      1440
   weighted avg       0.92      0.92      0.92      1440



**Many Layers**

In [ ]:
model = Sequential()
model.add(Dense(1024,input_dim=256*256*3, activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 1024)              201327616 
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_12 (Dense)             (None, 64)               

**Training Model**

In [ ]:
# Train model on dataset
#model.fit_generator(generator=training_generator,validation_data=validation_generator,use_multiprocessing=True, workers=6,epoch)
#model.fit_generator(generator=training_generator,validation_data=validation_generator,epochs=20)
training_loop(20,32,training_generator,validation_generator)



Start of epoch  0
Training acc over epoch: 0.8215
Loss value over epoch: 0.4517
Validation acc: 0.8743


Start of epoch  1
Training acc over epoch: 0.8894
Loss value over epoch: 0.3141
Validation acc: 0.8535


Start of epoch  2
Training acc over epoch: 0.9176
Loss value over epoch: 0.2302
Validation acc: 0.8889


Start of epoch  3
Training acc over epoch: 0.9324
Loss value over epoch: 0.1893
Validation acc: 0.8958


Start of epoch  4
Training acc over epoch: 0.9347
Loss value over epoch: 0.1722
Validation acc: 0.8778


Start of epoch  5
Training acc over epoch: 0.9484
Loss value over epoch: 0.1491
Validation acc: 0.9125


Start of epoch  6
Training acc over epoch: 0.9542
Loss value over epoch: 0.1247
Validation acc: 0.8882


Start of epoch  7
Training acc over epoch: 0.9609
Loss value over epoch: 0.1151
Validation acc: 0.8771


Start of epoch  8
Training acc over epoch: 0.9486
Loss value over epoch: 0.1430
Validation acc: 0.9174


Start of epoch  9
Training acc over epoch: 0.9609
Los

**Testing Model**

In [ ]:
#test_generator = DataGenerator(partition['test'], label_vector, 32, 256*256, 3, False, shuffle=False)
test_model(test_generator, Y_test)

confusion_matrix
[[463   6  11]
 [  7 412  61]
 [ 10  23 447]]

Accuracy: 0.92

Weighted Precision: 0.92
Weighted Recall: 0.92
Weighted F1-score: 0.92

Classification Report

                 precision    recall  f1-score   support

          Covid       0.96      0.96      0.96       480
         Normal       0.93      0.86      0.89       480
Viral Pneumonia       0.86      0.93      0.89       480

       accuracy                           0.92      1440
      macro avg       0.92      0.92      0.92      1440
   weighted avg       0.92      0.92      0.92      1440




**CNN**

In [ ]:
training_generator = DataGenerator(partition['train'], label_vector, 32, (256, 256), 3, True,3)
validation_generator = DataGenerator(partition['validation'], label_vector, 32, (256, 256), 3, True,3)
test_generator = DataGenerator(partition['test'], label_vector, 32, (256, 256), 3, True,3)

**Few Convolution Layers**

In [ ]:
model =Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
flatten (Flatten)            (None, 1000000)           0         
_________________________________________________________________
dense_21 (Dense)             (None, 32)                32000032  
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 99        
Total params: 32,019,523
Trainable params: 32,019,523
Non-trainable params: 0
__________________________________________

**Train the model**

In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.8132
Loss value over epoch: 0.4847
Validation acc: 0.9014


Start of epoch  1
Training acc over epoch: 0.9354
Loss value over epoch: 0.1963
Validation acc: 0.9375


Start of epoch  2
Training acc over epoch: 0.9549
Loss value over epoch: 0.1315
Validation acc: 0.9375


Start of epoch  3
Training acc over epoch: 0.9722
Loss value over epoch: 0.0897
Validation acc: 0.9479


Start of epoch  4
Training acc over epoch: 0.9810
Loss value over epoch: 0.0666
Validation acc: 0.9389


Start of epoch  5
Training acc over epoch: 0.9856
Loss value over epoch: 0.0510
Validation acc: 0.9410


Start of epoch  6
Training acc over epoch: 0.9896
Loss value over epoch: 0.0387
Validation acc: 0.9521


Start of epoch  7
Training acc over epoch: 0.9940
Loss value over epoch: 0.0250
Validation acc: 0.9493


Start of epoch  8
Training acc over epoch: 0.9968
Loss value over epoch: 0.0188
Validation acc: 0.9507


Start of epoch  9
Training acc over epoch: 0.9979
Los

**Test the model**

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[473   1   6]
 [  4 457  19]
 [ 14  20 446]]

Accuracy: 0.96

Weighted Precision: 0.96
Weighted Recall: 0.96
Weighted F1-score: 0.96

Classification Report

                 precision    recall  f1-score   support

          Covid       0.96      0.99      0.97       480
         Normal       0.96      0.95      0.95       480
Viral Pneumonia       0.95      0.93      0.94       480

       accuracy                           0.96      1440
      macro avg       0.96      0.96      0.96      1440
   weighted avg       0.96      0.96      0.96      1440



**Many Convolution Layers**

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 60, 60, 128)       147584    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 128)      

**Train the model**

In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.8160
Loss value over epoch: 0.4605
Validation acc: 0.9174


Start of epoch  1
Training acc over epoch: 0.9169
Loss value over epoch: 0.2258
Validation acc: 0.9417


Start of epoch  2
Training acc over epoch: 0.9444
Loss value over epoch: 0.1646
Validation acc: 0.9431


Start of epoch  3
Training acc over epoch: 0.9535
Loss value over epoch: 0.1374
Validation acc: 0.9458


Start of epoch  4
Training acc over epoch: 0.9657
Loss value over epoch: 0.0999
Validation acc: 0.9500


Start of epoch  5
Training acc over epoch: 0.9745
Loss value over epoch: 0.0780
Validation acc: 0.9431


Start of epoch  6
Training acc over epoch: 0.9775
Loss value over epoch: 0.0701
Validation acc: 0.9021


Start of epoch  7
Training acc over epoch: 0.9757
Loss value over epoch: 0.0766
Validation acc: 0.9062


Start of epoch  8
Training acc over epoch: 0.9775
Loss value over epoch: 0.0637
Validation acc: 0.9465


Start of epoch  9
Training acc over epoch: 0.9817
Los

**Testing Model**

In [ ]:
test_model(test_generator,Y_test)

confusion_matrix
[[470   4   6]
 [  6 464  10]
 [ 14  29 437]]

Accuracy: 0.95

Weighted Precision: 0.95
Weighted Recall: 0.95
Weighted F1-score: 0.95

Classification Report

                 precision    recall  f1-score   support

          Covid       0.96      0.98      0.97       480
         Normal       0.93      0.97      0.95       480
Viral Pneumonia       0.96      0.91      0.94       480

       accuracy                           0.95      1440
      macro avg       0.95      0.95      0.95      1440
   weighted avg       0.95      0.95      0.95      1440



**Pretrained Models Without Pretrained Weights**

**ResNet50**

In [ ]:
base_model = tf.keras.applications.ResNet50(
                include_top=False,
                weights=None,
                input_tensor=None,
                input_shape=(256,256,3),
                pooling='avg',
                classes=3,
                classifier_activation="softmax",
                )

base_model.trainable = False
inputs = keras.Input(shape=(256, 256,3))
x = base_model(inputs, training=False)
outputs = keras.layers.Dense(3 ,activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 6147      
Total params: 23,593,859
Trainable params: 6,147
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.3333
Loss value over epoch: 1.0973
Validation acc: 0.3333


Start of epoch  1
Training acc over epoch: 0.3333
Loss value over epoch: 1.0955
Validation acc: 0.3333


Start of epoch  2
Training acc over epoch: 0.3333
Loss value over epoch: 1.0942
Validation acc: 0.3333


Start of epoch  3
Training acc over epoch: 0.3333
Loss value over epoch: 1.0929
Validation acc: 0.3333


Start of epoch  4
Training acc over epoch: 0.3333
Loss value over epoch: 1.0915
Validation acc: 0.3333


Start of epoch  5
Training acc over epoch: 0.3333
Loss value over epoch: 1.0902
Validation acc: 0.3333


Start of epoch  6
Training acc over epoch: 0.3333
Loss value over epoch: 1.0890
Validation acc: 0.3333


Start of epoch  7
Training acc over epoch: 0.3333
Loss value over epoch: 1.0877
Validation acc: 0.3333


Start of epoch  8
Training acc over epoch: 0.3338
Loss value over epoch: 1.0864
Validation acc: 0.3333


Start of epoch  9
Training acc over epoch: 0.3361
Los

In [ ]:
base_model.trainable = True
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 6147      
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.6211
Loss value over epoch: 0.7935
Validation acc: 0.8076


Start of epoch  1
Training acc over epoch: 0.8278
Loss value over epoch: 0.4731
Validation acc: 0.8528


Start of epoch  2
Training acc over epoch: 0.8664
Loss value over epoch: 0.3873
Validation acc: 0.8847


Start of epoch  3
Training acc over epoch: 0.8854
Loss value over epoch: 0.3249
Validation acc: 0.9049


Start of epoch  4
Training acc over epoch: 0.9118
Loss value over epoch: 0.2532
Validation acc: 0.9236


Start of epoch  5
Training acc over epoch: 0.9275
Loss value over epoch: 0.2004
Validation acc: 0.9306


Start of epoch  6
Training acc over epoch: 0.9426
Loss value over epoch: 0.1718
Validation acc: 0.9375


Start of epoch  7
Training acc over epoch: 0.9479
Loss value over epoch: 0.1490
Validation acc: 0.9410


Start of epoch  8
Training acc over epoch: 0.9514
Loss value over epoch: 0.1435
Validation acc: 0.9333


Start of epoch  9
Training acc over epoch: 0.9618
Los

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[469   2   9]
 [  2 407  71]
 [  8  12 460]]

Accuracy: 0.93

Weighted Precision: 0.93
Weighted Recall: 0.93
Weighted F1-score: 0.93

Classification Report

                 precision    recall  f1-score   support

          Covid       0.98      0.98      0.98       480
         Normal       0.97      0.85      0.90       480
Viral Pneumonia       0.85      0.96      0.90       480

       accuracy                           0.93      1440
      macro avg       0.93      0.93      0.93      1440
   weighted avg       0.93      0.93      0.93      1440



**Inception V3**

In [ ]:
base_model = tf.keras.applications.InceptionV3(
                include_top=False,
                weights=None,
                input_tensor=None,
                input_shape=(256,256,3),
                pooling='avg',
                classes=3,
                classifier_activation="softmax",
               )

    
base_model.trainable = False
inputs = keras.Input(shape=(256, 256,3))
x = base_model(inputs, training=False)
outputs = keras.layers.Dense(3 ,activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 6147      
Total params: 21,808,931
Trainable params: 6,147
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.3322
Loss value over epoch: 1.0985
Validation acc: 0.3333


Start of epoch  1
Training acc over epoch: 0.3333
Loss value over epoch: 1.0984
Validation acc: 0.3333


Start of epoch  2
Training acc over epoch: 0.3333
Loss value over epoch: 1.0982
Validation acc: 0.3333


Start of epoch  3
Training acc over epoch: 0.3333
Loss value over epoch: 1.0981
Validation acc: 0.3333


Start of epoch  4
Training acc over epoch: 0.3333
Loss value over epoch: 1.0979
Validation acc: 0.3333


Start of epoch  5
Training acc over epoch: 0.3333
Loss value over epoch: 1.0978
Validation acc: 0.3333


Start of epoch  6
Training acc over epoch: 0.3333
Loss value over epoch: 1.0976
Validation acc: 0.3333


Start of epoch  7
Training acc over epoch: 0.3333
Loss value over epoch: 1.0975
Validation acc: 0.3333


Start of epoch  8
Training acc over epoch: 0.3333
Loss value over epoch: 1.0974
Validation acc: 0.3333


Start of epoch  9
Training acc over epoch: 0.3333
Los

In [ ]:
base_model.trainable = True
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 6147      
Total params: 21,808,931
Trainable params: 21,774,499
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.5995
Loss value over epoch: 0.8463
Validation acc: 0.8069


Start of epoch  1
Training acc over epoch: 0.8053
Loss value over epoch: 0.5227
Validation acc: 0.8562


Start of epoch  2
Training acc over epoch: 0.8495
Loss value over epoch: 0.4180
Validation acc: 0.8667


Start of epoch  3
Training acc over epoch: 0.8898
Loss value over epoch: 0.3242
Validation acc: 0.8993


Start of epoch  4
Training acc over epoch: 0.9123
Loss value over epoch: 0.2618
Validation acc: 0.9243


Start of epoch  5
Training acc over epoch: 0.9262
Loss value over epoch: 0.2333
Validation acc: 0.9236


Start of epoch  6
Training acc over epoch: 0.9292
Loss value over epoch: 0.2115
Validation acc: 0.9236


Start of epoch  7
Training acc over epoch: 0.9287
Loss value over epoch: 0.2126
Validation acc: 0.9229


Start of epoch  8
Training acc over epoch: 0.9384
Loss value over epoch: 0.1842
Validation acc: 0.9250


Start of epoch  9
Training acc over epoch: 0.9382
Los

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[475   5   0]
 [  4 471   5]
 [ 17  43 420]]

Accuracy: 0.95

Weighted Precision: 0.95
Weighted Recall: 0.95
Weighted F1-score: 0.95

Classification Report

                 precision    recall  f1-score   support

          Covid       0.96      0.99      0.97       480
         Normal       0.91      0.98      0.94       480
Viral Pneumonia       0.99      0.88      0.93       480

       accuracy                           0.95      1440
      macro avg       0.95      0.95      0.95      1440
   weighted avg       0.95      0.95      0.95      1440



**DenseNet**

In [ ]:
base_model = tf.keras.applications.DenseNet121(
                include_top=False,
                weights=None,
                input_tensor=None,
                input_shape=(256,256,3),
                pooling='avg',
                classes=3,
                )

base_model.trainable = False
inputs = keras.Input(shape=(256, 256,3))
x = base_model(inputs, training=False)
outputs = keras.layers.Dense(3 ,activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, 1024)              7037504   
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 3075      
Total params: 7,040,579
Trainable params: 3,075
Non-trainable params: 7,037,504
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.3669
Loss value over epoch: 1.0976
Validation acc: 0.3938


Start of epoch  1
Training acc over epoch: 0.3516
Loss value over epoch: 1.0957
Validation acc: 0.3333


Start of epoch  2
Training acc over epoch: 0.3333
Loss value over epoch: 1.0941
Validation acc: 0.3333


Start of epoch  3
Training acc over epoch: 0.3333
Loss value over epoch: 1.0927
Validation acc: 0.3333


Start of epoch  4
Training acc over epoch: 0.3333
Loss value over epoch: 1.0913
Validation acc: 0.3340


Start of epoch  5
Training acc over epoch: 0.3333
Loss value over epoch: 1.0900
Validation acc: 0.3340


Start of epoch  6
Training acc over epoch: 0.3336
Loss value over epoch: 1.0887
Validation acc: 0.3354


Start of epoch  7
Training acc over epoch: 0.3356
Loss value over epoch: 1.0874
Validation acc: 0.3382


Start of epoch  8
Training acc over epoch: 0.3396
Loss value over epoch: 1.0861
Validation acc: 0.3424


Start of epoch  9
Training acc over epoch: 0.3530
Los

In [ ]:
base_model.trainable = True
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, 1024)              7037504   
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 3075      
Total params: 7,040,579
Trainable params: 6,956,931
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.6738
Loss value over epoch: 0.7262
Validation acc: 0.8424


Start of epoch  1
Training acc over epoch: 0.8375
Loss value over epoch: 0.4443
Validation acc: 0.8785


Start of epoch  2
Training acc over epoch: 0.8722
Loss value over epoch: 0.3679
Validation acc: 0.8875


Start of epoch  3
Training acc over epoch: 0.8928
Loss value over epoch: 0.3147
Validation acc: 0.9208


Start of epoch  4
Training acc over epoch: 0.9067
Loss value over epoch: 0.2699
Validation acc: 0.9299


Start of epoch  5
Training acc over epoch: 0.9118
Loss value over epoch: 0.2531
Validation acc: 0.9312


Start of epoch  6
Training acc over epoch: 0.9174
Loss value over epoch: 0.2432
Validation acc: 0.9187


Start of epoch  7
Training acc over epoch: 0.9278
Loss value over epoch: 0.2142
Validation acc: 0.9292


Start of epoch  8
Training acc over epoch: 0.9326
Loss value over epoch: 0.1948
Validation acc: 0.9306


Start of epoch  9
Training acc over epoch: 0.9329
Los

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[470   5   5]
 [  2 449  29]
 [ 12  16 452]]

Accuracy: 0.95

Weighted Precision: 0.95
Weighted Recall: 0.95
Weighted F1-score: 0.95

Classification Report

                 precision    recall  f1-score   support

          Covid       0.97      0.98      0.98       480
         Normal       0.96      0.94      0.95       480
Viral Pneumonia       0.93      0.94      0.94       480

       accuracy                           0.95      1440
      macro avg       0.95      0.95      0.95      1440
   weighted avg       0.95      0.95      0.95      1440



**Pretrained Models Using pretrained Weights**

**ResNet50**

In [ ]:
base_model =tf.keras.applications.ResNet50(
                include_top=False,
                weights="imagenet",
                input_tensor=None,
                input_shape=(256,256,3),
                pooling='avg',
                classes=3,
                classifier_activation="softmax",
    
                )
base_model.trainable = False
inputs = keras.Input(shape=(256, 256,3))
x = base_model(inputs, training=False)
outputs = keras.layers.Dense(3 ,activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_32 (Dense)             (None, 3)                 6147      
Total params: 23,593,859
Trainable params: 6,147
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.4255
Loss value over epoch: 1.0987
Validation acc: 0.6528


Start of epoch  1
Training acc over epoch: 0.6634
Loss value over epoch: 0.9993
Validation acc: 0.6993


Start of epoch  2
Training acc over epoch: 0.7009
Loss value over epoch: 0.9321
Validation acc: 0.7215


Start of epoch  3
Training acc over epoch: 0.7204
Loss value over epoch: 0.8798
Validation acc: 0.7340


Start of epoch  4
Training acc over epoch: 0.7326
Loss value over epoch: 0.8386
Validation acc: 0.7354


Start of epoch  5
Training acc over epoch: 0.7417
Loss value over epoch: 0.8055
Validation acc: 0.7403


Start of epoch  6
Training acc over epoch: 0.7461
Loss value over epoch: 0.7786
Validation acc: 0.7431


Start of epoch  7
Training acc over epoch: 0.7507
Loss value over epoch: 0.7562
Validation acc: 0.7493


Start of epoch  8
Training acc over epoch: 0.7532
Loss value over epoch: 0.7373
Validation acc: 0.7569


Start of epoch  9
Training acc over epoch: 0.7542
Los

In [ ]:
base_model.trainable = True
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_32 (Dense)             (None, 3)                 6147      
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
training_loop(20,32,training_generator,validation_generator)



Start of epoch  0
Training acc over epoch: 0.3303
Loss value over epoch: 1.2439
Validation acc: 0.3333


Start of epoch  1
Training acc over epoch: 0.3289
Loss value over epoch: 1.1010
Validation acc: 0.3333


Start of epoch  2
Training acc over epoch: 0.5102
Loss value over epoch: 0.9379
Validation acc: 0.7208


Start of epoch  3
Training acc over epoch: 0.8111
Loss value over epoch: 0.4587
Validation acc: 0.8229


Start of epoch  4
Training acc over epoch: 0.8875
Loss value over epoch: 0.2924
Validation acc: 0.8944


Start of epoch  5
Training acc over epoch: 0.9141
Loss value over epoch: 0.2279
Validation acc: 0.9208


Start of epoch  6
Training acc over epoch: 0.9340
Loss value over epoch: 0.1793
Validation acc: 0.9292


Start of epoch  7
Training acc over epoch: 0.9370
Loss value over epoch: 0.1678
Validation acc: 0.9160


Start of epoch  8
Training acc over epoch: 0.9512
Loss value over epoch: 0.1376
Validation acc: 0.9236


Start of epoch  9
Training acc over epoch: 0.9611
Los

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[466  13   1]
 [  5 463  12]
 [ 33  47 400]]

Accuracy: 0.92

Weighted Precision: 0.93
Weighted Recall: 0.92
Weighted F1-score: 0.92

Classification Report

                 precision    recall  f1-score   support

          Covid       0.92      0.97      0.95       480
         Normal       0.89      0.96      0.92       480
Viral Pneumonia       0.97      0.83      0.90       480

       accuracy                           0.92      1440
      macro avg       0.93      0.92      0.92      1440
   weighted avg       0.93      0.92      0.92      1440



**InceptionV3**

In [ ]:
base_model =tf.keras.applications.InceptionV3(
                include_top=False,
                weights="imagenet",
                input_tensor=None,
                input_shape=(256,256,3),
                pooling='avg',
                classes=3,
                classifier_activation="softmax",
               )

    
base_model.trainable = False
inputs = keras.Input(shape=(256, 256,3))
x = base_model(inputs, training=False)
outputs = keras.layers.Dense(3 ,activation='softmax')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense_33 (Dense)             (None, 3)                 6147      
Total params: 21,808,931
Trainable params: 6,147
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.5979
Loss value over epoch: 0.8740
Validation acc: 0.7854


Start of epoch  1
Training acc over epoch: 0.8197
Loss value over epoch: 0.5370
Validation acc: 0.8514


Start of epoch  2
Training acc over epoch: 0.8738
Loss value over epoch: 0.4284
Validation acc: 0.8813


Start of epoch  3
Training acc over epoch: 0.8907
Loss value over epoch: 0.3711
Validation acc: 0.8875


Start of epoch  4
Training acc over epoch: 0.9019
Loss value over epoch: 0.3346
Validation acc: 0.8958


Start of epoch  5
Training acc over epoch: 0.9095
Loss value over epoch: 0.3086
Validation acc: 0.9035


Start of epoch  6
Training acc over epoch: 0.9141
Loss value over epoch: 0.2888
Validation acc: 0.9062


Start of epoch  7
Training acc over epoch: 0.9183
Loss value over epoch: 0.2728
Validation acc: 0.9097


Start of epoch  8
Training acc over epoch: 0.9234
Loss value over epoch: 0.2594
Validation acc: 0.9125


Start of epoch  9
Training acc over epoch: 0.9257
Los

In [ ]:
base_model.trainable = True
model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
dense_33 (Dense)             (None, 3)                 6147      
Total params: 21,808,931
Trainable params: 21,774,499
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.9051
Loss value over epoch: 0.2628
Validation acc: 0.9681


Start of epoch  1
Training acc over epoch: 0.9618
Loss value over epoch: 0.1110
Validation acc: 0.9514


Start of epoch  2
Training acc over epoch: 0.9799
Loss value over epoch: 0.0598
Validation acc: 0.9396


Start of epoch  3
Training acc over epoch: 0.9838
Loss value over epoch: 0.0486
Validation acc: 0.9625


Start of epoch  4
Training acc over epoch: 0.9794
Loss value over epoch: 0.0630
Validation acc: 0.9722


Start of epoch  5
Training acc over epoch: 0.9861
Loss value over epoch: 0.0442
Validation acc: 0.9229


Start of epoch  6
Training acc over epoch: 0.9863
Loss value over epoch: 0.0376
Validation acc: 0.9306


Start of epoch  7
Training acc over epoch: 0.9894
Loss value over epoch: 0.0287
Validation acc: 0.9785


Start of epoch  8
Training acc over epoch: 0.9947
Loss value over epoch: 0.0161
Validation acc: 0.9799


Start of epoch  9
Training acc over epoch: 0.9905
Los

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[477   1   2]
 [  0 478   2]
 [  0  13 467]]

Accuracy: 0.99

Weighted Precision: 0.99
Weighted Recall: 0.99
Weighted F1-score: 0.99

Classification Report

                 precision    recall  f1-score   support

          Covid       1.00      0.99      1.00       480
         Normal       0.97      1.00      0.98       480
Viral Pneumonia       0.99      0.97      0.98       480

       accuracy                           0.99      1440
      macro avg       0.99      0.99      0.99      1440
   weighted avg       0.99      0.99      0.99      1440



**DenseNet**

In [ ]:
base_model = tf.keras.applications.DenseNet121(
                include_top=False,
                weights="imagenet",
                input_tensor=None,
                input_shape=(256,256,3),
                pooling='avg',
                classes=3,
                )

base_model.trainable = False
inputs = keras.Input(shape=(256, 256,3))
x = base_model(inputs, training=False)
outputs = keras.layers.Dense(3 ,activation='softmax')(x)
model = keras.Model(inputs, outputs)

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, 1024)              7037504   
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 7,040,579
Trainable params: 3,075
Non-trainable params: 7,037,504
_________________________________________________________________


In [ ]:
training_loop(20, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.4560
Loss value over epoch: 1.1281
Validation acc: 0.6354


Start of epoch  1
Training acc over epoch: 0.7407
Loss value over epoch: 0.7432
Validation acc: 0.8229


Start of epoch  2
Training acc over epoch: 0.8454
Loss value over epoch: 0.5598
Validation acc: 0.8833


Start of epoch  3
Training acc over epoch: 0.8900
Loss value over epoch: 0.4499
Validation acc: 0.9076


Start of epoch  4
Training acc over epoch: 0.9120
Loss value over epoch: 0.3790
Validation acc: 0.9187


Start of epoch  5
Training acc over epoch: 0.9252
Loss value over epoch: 0.3304
Validation acc: 0.9285


Start of epoch  6
Training acc over epoch: 0.9317
Loss value over epoch: 0.2952
Validation acc: 0.9333


Start of epoch  7
Training acc over epoch: 0.9350
Loss value over epoch: 0.2688
Validation acc: 0.9361


Start of epoch  8
Training acc over epoch: 0.9373
Loss value over epoch: 0.2482
Validation acc: 0.9361


Start of epoch  9
Training acc over epoch: 0.9403
Los

In [ ]:
base_model.trainable = True
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
densenet121 (Functional)     (None, 1024)              7037504   
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 7,040,579
Trainable params: 6,956,931
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
training_loop(10, 32, training_generator, validation_generator)



Start of epoch  0
Training acc over epoch: 0.9174
Loss value over epoch: 0.2646
Validation acc: 0.9590


Start of epoch  1
Training acc over epoch: 0.9639
Loss value over epoch: 0.1094
Validation acc: 0.9500


Start of epoch  2
Training acc over epoch: 0.9741
Loss value over epoch: 0.0717
Validation acc: 0.9674


Start of epoch  3
Training acc over epoch: 0.9854
Loss value over epoch: 0.0439
Validation acc: 0.9799


Start of epoch  4
Training acc over epoch: 0.9863
Loss value over epoch: 0.0375
Validation acc: 0.9590


Start of epoch  5
Training acc over epoch: 0.9850
Loss value over epoch: 0.0413
Validation acc: 0.9722


Start of epoch  6
Training acc over epoch: 0.9850
Loss value over epoch: 0.0394
Validation acc: 0.9743


Start of epoch  7
Training acc over epoch: 0.9845
Loss value over epoch: 0.0423
Validation acc: 0.9743


Start of epoch  8
Training acc over epoch: 0.9875
Loss value over epoch: 0.0324
Validation acc: 0.9833


Start of epoch  9
Training acc over epoch: 0.9937
Los

In [ ]:
test_model(test_generator, Y_test)

confusion_matrix
[[480   0   0]
 [  1 479   0]
 [  6  21 453]]

Accuracy: 0.98

Weighted Precision: 0.98
Weighted Recall: 0.98
Weighted F1-score: 0.98

Classification Report

                 precision    recall  f1-score   support

          Covid       0.99      1.00      0.99       480
         Normal       0.96      1.00      0.98       480
Viral Pneumonia       1.00      0.94      0.97       480

       accuracy                           0.98      1440
      macro avg       0.98      0.98      0.98      1440
   weighted avg       0.98      0.98      0.98      1440





**Using different split values**

**Splitting ratio: (80:10:10)**

In [ ]:
# split data into train and test (85:15)
X_train, X_test, Y_train, Y_test = train_test_split(data, label_vector, test_size=0.15, random_state=1, stratify=label_vector)

# split train data into train and validation (88.9:11.1)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.1764, random_state=1, stratify=Y_train)

print(X_train.shape)
print(X_validation.shape)
print(X_test.shape)

(5040,)
(1080,)
(1080,)


In [ ]:
partition = {
    "train": X_train,
    "validation": X_validation,
    "test": X_test
}

In [ ]:
training_generator = DataGenerator(partition['train'], label_vector, 32, (256, 256), 3, True,3,shuffle=True)
validation_generator = DataGenerator(partition['validation'], label_vector, 32, (256, 256), 3, True,3,shuffle=True)
test_generator = DataGenerator(partition['test'], label_vector, 32, (256, 256), 3, True,3,shuffle=True)

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 125, 125, 128)     73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 60, 60, 128)       147584    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 28, 28, 128)      

In [ ]:
training_loop(20,32,training_generator,validation_generator)



Start of epoch  0
Training acc over epoch: 0.8246
Loss value over epoch: 0.4485
Validation acc: 0.8944


Start of epoch  1
Training acc over epoch: 0.9208
Loss value over epoch: 0.2226
Validation acc: 0.9306


Start of epoch  2
Training acc over epoch: 0.9393
Loss value over epoch: 0.1658
Validation acc: 0.9426


Start of epoch  3
Training acc over epoch: 0.9558
Loss value over epoch: 0.1255
Validation acc: 0.9324


Start of epoch  4
Training acc over epoch: 0.9633
Loss value over epoch: 0.1037
Validation acc: 0.9398


Start of epoch  5
Training acc over epoch: 0.9726
Loss value over epoch: 0.0811
Validation acc: 0.9426


Start of epoch  6
Training acc over epoch: 0.9788
Loss value over epoch: 0.0666
Validation acc: 0.9463


Start of epoch  7
Training acc over epoch: 0.9839
Loss value over epoch: 0.0507
Validation acc: 0.9528


Start of epoch  8
Training acc over epoch: 0.9835
Loss value over epoch: 0.0442
Validation acc: 0.9509


Start of epoch  9
Training acc over epoch: 0.9889
Los

In [ ]:
#test_model(test_generator,Y_test)
y_truth = []
y_hat = []
num_batches_test = test_generator.__len__()
for j in range(num_batches_test):
  x,y = test_generator.__getitem__(j)
  #print(x.shape,y.shape)
  y_pred = model.predict(x)
  for i in range(len(y)):
    max_number_truth = max(y[i])
    index_of_maximum_truth = np.where(y[i] == max_number_truth)
    y_truth.append(index_of_maximum_truth[0][0])
    max_number_pred = max(y_pred[i])
    index_of_maximum_pred = np.where(y_pred[i] == max_number_pred)
    y_hat.append(index_of_maximum_pred[0][0])
    
print(report(y_hat,y_truth))

confusion_matrix
[[355   1   4]
 [  2 349   9]
 [  9  18 333]]

Accuracy: 0.96

Weighted Precision: 0.96
Weighted Recall: 0.96
Weighted F1-score: 0.96

Classification Report

                 precision    recall  f1-score   support

          Covid       0.97      0.99      0.98       360
         Normal       0.95      0.97      0.96       360
Viral Pneumonia       0.96      0.93      0.94       360

       accuracy                           0.96      1080
      macro avg       0.96      0.96      0.96      1080
   weighted avg       0.96      0.96      0.96      1080

None
